In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from scipy.stats.stats import pearsonr

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-data/"))

['test_df_h.pkl', 'test_df_c.pkl', 'train_df_w.pkl', 'test_df_w.pkl', 'train_df_s.pkl', 'test_df_s.pkl', 'test_df_r.pkl', 'train_df_h.pkl', 'test_identity.pkl', 'train_df_r.pkl', 'train_identity.pkl', 'train_df_c.pkl']


In [2]:
train_df = pd.read_pickle('../input/mini-model-data/train_df_w.pkl')
test_df = pd.read_pickle('../input/mini-model-data/test_df_w.pkl')

train_identity = pd.read_pickle('../input/mini-model-data/train_identity.pkl')
test_identity = pd.read_pickle('../input/mini-model-data/test_identity.pkl')

train = train_df.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_df.merge(test_identity, how='left', left_index=True, right_index=True)

train_y = train_df['isFraud'].copy()

# print(train_df.shape)
# print(test_df.shape)
# print(train_identity.shape)
# print(test_identity.shape)
print(train.shape)
print(test.shape)

del train_df, test_df, train_identity, test_identity

(439670, 433)
(360987, 433)


In [3]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1

In [4]:
target_encode_book = [
    'addr1', 'C1', 'C2', 'C5','C6', 'C9', 'C11', 'C13', 'C14', 'M4',
    'card1', 'card2', 'card3', 'card4', 'card5','card6',
    'M1','M2','M3','M4','M5','M6','M7','M8','M9',
    'P_emaildomain','cents',
    'DT_hour', 'DT_day_week', 'DT_day'
]
print('target book length is:', len(target_encode_book))

target book length is: 30


In [5]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1

In [6]:
def fe_dt(df):
    
    df['DT'] = df['TransactionDT'].apply(
        lambda x: (START_DATE + datetime.timedelta(seconds=x)))  # tmp    
    df['DT_month'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
    df['DT_week_year'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
    df['DT_day_year'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear
    
    df['DT_hour'] = df['DT'].dt.hour
    df['DT_day_week'] = df['DT'].dt.dayofweek
    df['DT_day'] = df['DT'].dt.day
    
    for col in ['D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15']:
        # day map
        day_temp_dict = df.groupby('DT_day_year')[col].agg(['mean']).rename(columns={'mean':col + '_day_mean'})
        day_temp_dict = day_temp_dict[col + '_day_mean'].to_dict()
        df[col + '_day_mean'] = df['DT_day_year'].map(day_temp_dict)
        df[col + '_day_mean'] = df[col] - df[col + '_day_mean']
        
        # week map
        week_temp_dict = df.groupby('DT_week_year')[col].agg(['mean']).rename(columns={'mean':col + '_week_mean'})
        week_temp_dict = week_temp_dict[col + '_week_mean'].to_dict()
        df[col + '_week_mean'] = df['DT_week_year'].map(week_temp_dict)
        df[col + '_week_mean'] = df[col] - df[col + '_week_mean']

    df = df.drop(['DT'],axis=1)
    
    return df

In [7]:
def fe_uid(train_df, test_df):
    
    train_df['uid'] = train_df['card1'].astype(str)+'_'+train_df['card2'].astype(str)
    test_df['uid'] = test_df['card1'].astype(str)+'_'+test_df['card2'].astype(str)

    train_df['uid2'] = train_df['uid'].astype(str)+'_'+train_df['card3'].astype(str)+'_'+train_df['card5'].astype(str)
    test_df['uid2'] = test_df['uid'].astype(str)+'_'+test_df['card3'].astype(str)+'_'+test_df['card5'].astype(str)

    train_df['uid3'] = train_df['uid2'].astype(str)+'_'+train_df['addr1'].astype(str)+'_'+train_df['addr2'].astype(str)
    test_df['uid3'] = test_df['uid2'].astype(str)+'_'+test_df['addr1'].astype(str)+'_'+test_df['addr2'].astype(str)

    train_df['uid4'] = train_df['card3'].astype(str)+'_'+train_df['P_emaildomain'].astype(str)
    test_df['uid4'] = test_df['card3'].astype(str)+'_'+test_df['P_emaildomain'].astype(str)
    
    # Check if the Transaction Amount is common or not (we can use freq encoding here)
    # In our dialog with a model we are telling to trust or not to these values   
    train_df['TransactionAmt_check'] = np.where(train_df['TransactionAmt'].isin(test_df['TransactionAmt']), 1, 0)
    test_df['TransactionAmt_check']  = np.where(test_df['TransactionAmt'].isin(train_df['TransactionAmt']), 1, 0)

    i_cols = ['card1','card2','card3','card5','uid','uid2','uid3', 'uid4']

    for col in i_cols:
        for agg_type in ['mean','std']:
            new_col_name = col+'_TransactionAmt_'+agg_type
            temp_df = pd.concat([train_df[[col, 'TransactionAmt']], test_df[[col,'TransactionAmt']]])
            #temp_df['TransactionAmt'] = temp_df['TransactionAmt'].astype(int)
            temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                    columns={agg_type: new_col_name})

            temp_df.index = list(temp_df[col])
            temp_df = temp_df[new_col_name].to_dict()   

            train_df[new_col_name] = train_df[col].map(temp_df)
            test_df[new_col_name]  = test_df[col].map(temp_df)

    train_df['TransactionAmt'] = np.log1p(train_df['TransactionAmt'])
    test_df['TransactionAmt'] = np.log1p(test_df['TransactionAmt'])   
    
#     train_df = train_df.drop(['uid','uid2','uid3', 'uid4'],axis=1)
#     test_df = test_df.drop(['uid','uid2','uid3', 'uid4'],axis=1)
    
    train_df = train_df.replace(np.inf, 999)
    test_df = test_df.replace(np.inf, 999)
    
    return train_df, test_df

In [8]:
# feature engineering

def fe(train, test):

    drop_na_cols = []
    for col in train.columns:
        if (len(train[col].dropna().unique()) <= 1):
            drop_na_cols.append(col)

    # remove columns
    train = train.drop(drop_na_cols, axis=1)
    test = test.drop(drop_na_cols, axis=1)
    
    # M features
    m_cols = ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9']
    for col in m_cols:
        train[col] = train[col].map(bool_map)
        test[col] = test[col].map(bool_map)
        
    for df in [train, test]:
        df['M_sum'] = df[m_cols].sum(axis=1).astype(np.int8)
        df['M_na'] = df[m_cols].isna().sum(axis=1).astype(np.int8)
        
#         # nan encoding
#         for col in train.columns:
#             if(train[col].isnull().sum() / len(train) > 0.2):
#                 df[col + '_isnull'] = df[col].isnull() + 0
    
    # user id
    train, test = fe_uid(train, test)

    ignore_cols = ['P_emaildomain']
    for col in train.columns:
        if (train[col].dtype == 'object' or test[col].dtype == 'object') and col not in ignore_cols:
            print('processing', col)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[col].values) + list(test[col].values))
            train[col] = lbl.transform(list(train[col].values))
            test[col] = lbl.transform(list(test[col].values))
    
    train['P_emaildomain'] = train['P_emaildomain'].map(emails)
    test['P_emaildomain'] = test['P_emaildomain'].map(emails)

    train['cents'] = train['TransactionAmt'].apply(lambda x: x - int(x))
    test['cents'] = test['TransactionAmt'].apply(lambda x: x - int(x))
    
    train = fe_dt(train)
    test = fe_dt(test)
    
    # add user id encoding
    uid_target_book = ['uid', 'uid2', 'uid3', 'uid4']
    uid_target_book = []
    for col in target_encode_book + uid_target_book:
        train[col+'_target_code'], test[col+'_target_code'] = target_encode(train[col], test[col], train_y,min_samples_leaf=100, noise_level=0.01)

        temp_df = pd.concat([train[[col]], test[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()   
        train[col+'_fq_enc'] = train[col].map(fq_encode)
        test[col+'_fq_enc']  = test[col].map(fq_encode)
        
        
    un_corr_cols = []
    for col in train.columns:
        if (train[col].dtype != 'object'):
#             print('processing', col)
            if(abs(pearsonr(train[col].fillna(-999), train_y)[1]) > 0.05 and col != 'DT_month'):
                un_corr_cols.append(col)
            print(col, pearsonr(train[col].fillna(-999), train_y))

    train = train.drop(target_encode_book + un_corr_cols, axis=1)
    test = test.drop(target_encode_book + un_corr_cols, axis=1)
    
    return train, test, drop_na_cols + un_corr_cols


train, test, drop_cols = fe(train, test)

processing card4
processing card6
processing M4
processing uid
processing uid2
processing uid3
processing uid4
isFraud (0.9999999999999731, 0.0)
TransactionDT (0.0013177220429272764, 0.38225551921427214)
TransactionAmt (0.06043518875696903, 0.0)
card1 (-0.0001802471556489793, 0.9048655719045225)
card2 (0.010550457787951759, 2.635015421329397e-12)
card3 (0.0020529816327271057, 0.173424983706016)
card4 (0.003348994311019993, 0.02637547522785591)
card5 (-0.007647757604433142, 3.955113998537005e-07)
card6 (-0.0656191912219369, 0.0)
addr1 (0.004535665356337477, 0.0026340741987864453)
addr2 (1.0142851598786374e-05, 0.9946338970118848)
dist1 (0.003857777849348453, 0.010527524921775992)
C1 (-0.017020759216639456, 1.5240937019268828e-29)
C2 (-0.015813889586842347, 9.971588635299631e-26)
C5 (-0.020651629874292168, 1.0872296305118369e-42)
C6 (-0.015971476962959762, 3.282398784728664e-26)
C9 (-0.015818418886858844, 9.659622237561824e-26)
C11 (-0.016109553744836178, 1.2288453649630305e-26)
C12 (0.0

V309 (0.06519224924833739, 0.0)
V310 (0.06259635271263286, 0.0)
V311 (0.013677110685016454, 1.197132122316395e-19)
V312 (0.10112069320042483, 0.0)
V313 (0.07468370416858451, 0.0)
V314 (0.08096314648593525, 0.0)
V315 (0.08618077342277536, 0.0)
V316 (0.06228916252611963, 0.0)
V317 (0.0891914869317508, 0.0)
V318 (0.09837894489345449, 0.0)
V319 (0.012250071474386259, 4.547540665565345e-16)
V320 (0.04545066178787317, 9.872955467597673e-200)
V321 (0.02862768492448197, 2.2192674636111663e-80)
M_sum (-0.010172307761731201, 1.528478132810334e-11)
M_na (-0.010657905529610216, 1.5810956076663579e-12)
uid (0.00497344049800372, 0.0009745215592214028)
uid2 (0.004971383322828025, 0.0009792662127221174)
uid3 (0.004941207323637798, 0.0010513675146575838)
uid4 (-0.0011772331019192356, 0.43504175669340267)
TransactionAmt_check (0.009636821347844157, 1.6580253197607147e-10)
card1_TransactionAmt_mean (0.06515383718923028, 0.0)
card1_TransactionAmt_std (0.052372968033063766, 1.3378262687839698e-264)
card2_T

In [9]:
# 210
print(train.shape)
print(test.shape)

(439670, 214)
(360987, 214)


In [10]:
# col = 'uid3_TransactionAmt_mean_target_code'
# plt.plot(train[train.isFraud==0][col],'.b')
# plt.plot(train[train.isFraud==1][col],'.r')

In [11]:
train.to_pickle('../input/mini-model-features/train_features_w.pkl')
test.to_pickle('../input/mini-model-features/test_features_w.pkl')

# POINT_SIZE = 2
# for col in train.columns:
#     fig = plt.figure(figsize=(12,8))

#     plt.scatter(train[train['DT_month'] == 12]['TransactionDT'], train[train['DT_month'] == 12][col], s=POINT_SIZE, c='b')     
#     plt.scatter(train[train['DT_month'] == 13]['TransactionDT'], train[train['DT_month'] == 13][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 14]['TransactionDT'], train[train['DT_month'] == 14][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 15]['TransactionDT'], train[train['DT_month'] == 15][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 16]['TransactionDT'], train[train['DT_month'] == 16][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 17]['TransactionDT'], train[train['DT_month'] == 17][col], s=POINT_SIZE, c='k')

#     plt.scatter(train[train.isFraud==1]['TransactionDT'], train[train.isFraud==1][col], s=POINT_SIZE * 3, c='r')
    
#     plt.scatter(test['TransactionDT'], test[col], s=POINT_SIZE, c='g')
#     plt.title(col)
#     plt.grid()
# #     plt.show()
    
#     fig.savefig("../eda/mini-models/W/W_" + str(col) + ".png")
#     plt.cla()
#     plt.clf()
#     plt.close(fig)
